# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [43]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,vershino-darasunskiy,52.3500,115.5167,-16.72,93,17,1.64,RU,1699734065
1,1,edinburgh of the seven seas,-37.0676,-12.3116,14.76,90,87,9.62,SH,1699733791
2,2,blackmans bay,-43.0167,147.3167,10.96,69,100,1.34,AU,1699734067
3,3,vilyuchinsk,52.9306,158.4028,2.15,73,59,19.66,RU,1699734068
4,4,puerto baquerizo moreno,-0.9000,-89.6000,28.55,61,40,6.69,EC,1699734069


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
map_plot_cities = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "EsriNatGeo",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    scale = 1,
    color = "City",
    alpha = 0.5
)

# Display the map
map_plot_cities

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [15]:
# Narrow down cities that fit criteria and drop any results with null values

city_weather_df = (city_data_df["Max Temp"] >= 21) & (city_data_df["Max Temp"] <= 30)
ideal_city = city_data_df[city_weather_df]


# Drop any rows with null values

ideal_weather_df = ideal_city.dropna()

# Display sample data
ideal_weather_df.head()


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
4,4,puerto baquerizo moreno,-0.9000,-89.6000,28.55,61,40,6.69,EC,1699734069
14,14,manokwari,-0.8667,134.0833,25.72,80,100,1.89,ID,1699734079
15,15,fale old settlement,-9.3852,-171.2468,28.61,74,94,3.22,TK,1699734079
21,21,lihue,21.9789,-159.3672,28.58,74,100,6.17,US,1699734085
29,29,saipan,15.1355,145.7010,28.39,78,40,6.69,MP,1699734092


### Step 3: Create a new DataFrame called `hotel_df`.

In [24]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity

keep_cells = ["City", "Country", "Lat", "Lng", "Humidity"]
hotel_df = ideal_weather_df[keep_cells].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
4,puerto baquerizo moreno,EC,-0.9000,-89.6000,61,
14,manokwari,ID,-0.8667,134.0833,80,
15,fale old settlement,TK,-9.3852,-171.2468,74,
21,lihue,US,21.9789,-159.3672,74,
29,saipan,MP,15.1355,145.7010,78,
...,...,...,...,...,...,...
583,meiganga,CM,6.5167,14.3000,73,
585,nadi,FJ,-17.8000,177.4167,69,
588,alaghsas,NE,17.0187,8.0168,15,
591,orkney,ZA,-26.9802,26.6727,47,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [49]:
# Set parameters to search for a hotel

radius = 10000

params = {
    "categories": "accommodation.hotel",
    "apiKey":geoapify_key,
    "limit": 20
      
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]

    filters = f"circle:{longitude},{latitude},{radius}"
    bias = f"proximity:{longitude},{latitude}"

    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = filters
    params["bias"] = bias
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make an API request using the params dictionary
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    #hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
       hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
       hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
    hotel_df

Starting hotel search
puerto baquerizo moreno - nearest hotel: Dory's House
manokwari - nearest hotel: Hotel Arfak
fale old settlement - nearest hotel: No hotel found
lihue - nearest hotel: Kauai Palms
saipan - nearest hotel: Chalan Kanoa Beach Hotel
warrenton - nearest hotel: No hotel found
bougouni - nearest hotel: Hotel Piemont
avarua - nearest hotel: Paradise Inn
dubbo - nearest hotel: Blue Gum Motel
puerto ayora - nearest hotel: Hostal La Mirada De Solitario George
flying fish cove - nearest hotel: Christmas Island Lodge
adamstown - nearest hotel: No hotel found
palikir - national government center - nearest hotel: Mangrove Bay Hotel
sumbe - nearest hotel: No hotel found
axim - nearest hotel: Axim Beach Hotel
georgetown - nearest hotel: Page 63 hostel
auki - nearest hotel: No hotel found
port mathurin - nearest hotel: Escale Vacances
west island - nearest hotel: Cocos Village Bungalows
sinabang - nearest hotel: No hotel found
palang - nearest hotel: No hotel found
la'ie - nearest 

### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [53]:
%%capture --no-display

# Configure the map plot
map_plot_hotels = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "EsriNatGeo",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    scale = 1,
    alpha = 0.5,
    color = "City",
    hover_cols = ["Country", "Hotel Name"]
)

# Display the map
map_plot_hotels


:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country,Hotel Name)